In [1]:
# Import standard libraries
import os
from contextlib import redirect_stdout

import sys
# append coeqwal packages to path
sys.path.append('./coeqwalpackage')

import numpy as np
import pandas as pd
import datetime as dt


In [2]:
# Import custom libraries
# Note: on my computer the next import doesn't work the first time I call it, why? If I re-run the cell, then it is ok. MUST DEBUG
from coeqwalpackage.metrics import *
import cqwlutils as cu
import plotting as pu

## Define contol file name

In [3]:
CtrlFile = 'CalSim3DataExtractionInitFile_v3.xlsx'
CtrlTab = 'Init'

## Read from control file

In [4]:
ScenarioListFile, ScenarioListTab, ScenarioListPath, DVDssNamesOutPath, SVDssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, DVDssMin, DVDssMax, SVDssMin, SVDssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax = cu.read_init_file(CtrlFile, CtrlTab)

### Read in Data

In [5]:
df, dss_names = read_in_df(ConvertDataOutPath,DVDssNamesOutPath)

In [6]:
df = add_water_year_column(df)

/Users/canruso/Desktop/COEQWAL_MAIN/coeqwal/notebooks/coeqwalpackage/metrics.py:175: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return df_copy.drop(["Date", "Year", "Month"], axis=1)


### Get metrics paths and make directories if needed

In [7]:
metrics_path = GroupDataDirPath + "/metrics_output"
if not os.path.exists(metrics_path):
    os.makedirs(metrics_path)

## Metrics Examples

In [8]:
# To get a single metric and variable dataframe, type metrics[Month_{var}_{metric}] ie metrics['Apr_S_RESTOT_NOD_mnth_avg'], thresholds["S_SHSTA_frequency_hitting_level5"]
# Month = Apr, Sept... or None
# Metric = [ann_avg, mnth_avg, compute_iqr, ann_percentile, moy_avgs, mnth_percentile, annual_totals]

In [9]:
# Thresholds

variables = ["S_SHSTA_", "S_OROVL_", "S_TRNTY_", "S_SLUIS_CVP_", "S_SLUIS_SWP_"]
variables_level1 = ["S_SHSTALEVEL1DV", "S_OROVLLEVEL1DV", "S_TRNTYLEVEL1DV", "S_SLUIS_CVPLEVEL1DV", "S_SLUIS_SWPLEVEL1DV"]
variables_level5 = ["S_SHSTALEVEL5DV", "S_OROVLLEVEL5DV", "S_TRNTYLEVEL5DV", "S_SLUIS_CVPLEVEL5DV", "S_SLUIS_SWPLEVEL5DV"]
write_out_dfs = []

In [10]:
thresholds = {}

for var, var_lvl1, var_lvl5 in zip(variables, variables_level1, variables_level5):
    # LEVEL5DV calculation (LEVEL5DV - var < 0)
    exceedance_days, exceedance_days_fraction = frequency_hitting_level(df, dss_names, var, var_lvl5, "TAF", f"All_Prob_{var}flood")
    thresholds[f"{var}frequency_hitting_level5"] = (exceedance_days, exceedance_days_fraction)

    # LEVEL1DV calculation (var - LEVEL1DV < 0)
    exceedance_days, exceedance_days_fraction = frequency_hitting_level(df, dss_names, var_lvl1, var, "TAF", f"All_Prob_{var}dead")
    thresholds[f"{var}frequency_hitting_level1"] = (exceedance_days, exceedance_days_fraction)

    # September LEVEL1DV calculation
    exceedance_days, exceedance_days_fraction = frequency_hitting_level(df, dss_names, var_lvl1, var, "TAF", f"Sept_Prob_{var}dead", months=[9])
    thresholds[f"Sept_{var}frequency_hitting_level1"] = (exceedance_days, exceedance_days_fraction)

for threshold, (threshold_df, threshold_fraction) in thresholds.items():
    write_out_dfs.append(threshold_fraction)
    display(threshold_fraction)

,All_Prob_S_SHSTA_flood
s0001,0.333333
s0002,0.315000
s0003,0.222500
s0004,0.192500
s0005,0.175833
s0006,0.303333
s0007,0.301667
s0008,0.300833
s0009,0.300833
s0010,0.301667


,All_Prob_S_SHSTA_dead
s0001,0.004167
s0002,0.017500
s0003,0.038333
s0004,0.052500
s0005,0.053333
s0006,0.003333
s0007,0.000000
s0008,0.000000
s0009,0.000000
s0010,0.000000


,Sept_Prob_S_SHSTA_dead
s0001,0.01
s0002,0.05
s0003,0.10
s0004,0.11
s0005,0.11
s0006,0.01
s0007,0.00
s0008,0.00
s0009,0.00
s0010,0.00


,All_Prob_S_OROVL_flood
s0001,0.215000
s0002,0.210000
s0003,0.165833
s0004,0.144167
s0005,0.133333
s0006,0.203333
s0007,0.201667
s0008,0.200000
s0009,0.201667
s0010,0.201667


,All_Prob_S_OROVL_dead
s0001,0.000000
s0002,0.000833
s0003,0.010000
s0004,0.016667
s0005,0.024167
s0006,0.000000
s0007,0.000000
s0008,0.000000
s0009,0.000000
s0010,0.000000


...

,All_Prob_S_SLUIS_CVP_dead
s0001,0.015833
s0002,0.027500
s0003,0.057500
s0004,0.085000
s0005,0.102500
s0006,0.017500
s0007,0.015000
s0008,0.017500
s0009,0.017500
s0010,0.017500


,Sept_Prob_S_SLUIS_CVP_dead
s0001,0.05
s0002,0.08
s0003,0.13
s0004,0.21
s0005,0.22
s0006,0.00
s0007,0.00
s0008,0.00
s0009,0.00
s0010,0.00


,All_Prob_S_SLUIS_SWP_flood
s0001,0.065833
s0002,0.073333
s0003,0.060000
s0004,0.051667
s0005,0.042500
s0006,0.057500
s0007,0.160000
s0008,0.180000
s0009,0.189167
s0010,0.190000


,All_Prob_S_SLUIS_SWP_dead
s0001,0.029167
s0002,0.026667
s0003,0.027500
s0004,0.055833
s0005,0.043333
s0006,0.040000
s0007,0.030833
s0008,0.033333
s0009,0.033333
s0010,0.033333


,Sept_Prob_S_SLUIS_SWP_dead
s0001,0.05
s0002,0.05
s0003,0.06
s0004,0.13
s0005,0.07
s0006,0.06
s0007,0.06
s0008,0.05
s0009,0.07
s0010,0.07


In [17]:
# Flow Requirements - only running scenario 18 (dss_names is messing it up, replace ["s0018"] with dss_names later)
# Missing C_SAC000_MIF_s0018

actual_flows = ["C_AMR004_s0018", "C_TRN111_s0018", "C_FTR029_s0018", "C_MOK028_s0018", "C_MCD005_s0018", "C_SAC257_s0018", "C_SJR127_s0018", "C_SAC289_s0018", "C_STS011_s0018",
                 "C_TUO003_s0018", "C_YUB002_s0018", "SP_SAC159_BTC003_s0018", "C_SAC148_s0018", "C_SAC122_s0018", "C_FTR003_s0018", "C_SAC049_s0018", "C_SJR070_s0018"
]
min_flows = ["C_AMR004_MIF_s0018", "C_TRN111_MIF_s0018", "C_FTR029_MIF_s0018", "C_MOK028_MIF_s0018", "C_MCD005_MIF_s0018", "C_SAC257_MIF_s0018", "C_SJR127_MIF_s0018", "C_SAC289_MIF_s0018", "C_STS011_MIF_s0018",
             "C_TUO003_MIF_s0018", "C_YUB002_MIF_s0018", "SP_SAC159_BTC003_MIF_s0018", "C_SAC148_MIF_s0018", "C_SAC122_MIF_s0018", "C_FTR003_MIF_s0018", "C_SAC049_MIF_s0018", "C_SJR070_MIF_s0018"
]


In [27]:
min_flows = [
    "C_AMR004", "C_TRN111", "C_FTR029", "C_MOK028", "C_MCD005",
    "C_SAC257", "C_SJR127", "C_SAC289", "C_STS011", "C_TUO003",
    "C_YUB002", "SP_SAC159_BTC003", "C_SAC148", "C_SAC122",
    "C_FTR003", "C_SAC049", "C_SJR070"
]

prob_result_list = []

for scenario in dss_names:
    scenario_short = scenario[:5]
    row_data = {"Scenario": scenario_short}

    for base_flow in min_flows:
        var1_name = f"{base_flow}_{scenario_short}"
        var2_name = f"{base_flow}_MIF_{scenario_short}"

        p_less = probability_var1_lt_var2_for_scenario(df, var1_name, var2_name, units="TAF", tolerance=1e-6)
        p_equal = probability_var1_eq_var2_for_scenario(df, var1_name, var2_name, units="TAF", tolerance=1e-6)

        lt_col = f"Prob_{base_flow}_LT_{base_flow}_MIF"
        eq_col = f"Prob_{base_flow}_EQ_{base_flow}_MIF"

        row_data[lt_col] = p_less
        row_data[eq_col] = p_equal

    prob_result_list.append(row_data)

df_flow_probs = pd.DataFrame(prob_result_list)
df_flow_probs.set_index("Scenario", inplace=True)

display(df_flow_probs)


[DEBUG MAIN] dss_names: ['s0001_DCR2023_9.3.1_danube_hist\\Model_Files\\DSS\\output\\DCR2023_DV_9.3.1_Danube_Hist_v1.7.dss', 's0002_DCR2023_9.3.1_danube_adj\\Model_Files\\DSS\\output\\DCR2023_DV_9.3.1_v2a_Danube_Adj_v1.8.dss', 's0003_DCR2023_9.3.1_danube_cc50\\Model_Files\\DSS\\output\\DCR2023_DV_9.3.1_Danube_cc50_v1.8.dss', 's0004_DCR2023_9.3.1_danube_cc75\\Model_Files\\DSS\\output\\DCR2023_DV_9.3.1_Danube_CC75_v1.8.dss', 's0005_DCR2023_9.3.1_danube_cc95\\Model_Files\\DSS\\output\\DCR2023_DV_9.3.1_Danube_CC95_v1.8.dss', 's0006_draftDCP_NDDOff_2020\\Model_Files\\DSS\\output\\0_DCR19_12.30_120621_NDDOff_2020.dss', 's0007_draftDCP_NDD_SWP3000_2020\\Model_Files\\DSS\\output\\1_DCR19_12.30_120621_NDD_SWP3000_2020.dss', 's0008_draftDCP_NDD_SWP4500_2020\\Model_Files\\DSS\\output\\2_DCR19_12.30_120621_NDD_SWP4500_2020.dss', 's0009_draftDCP_NDD_SWP6000_2020\\Model_Files\\DSS\\output\\3_DCR19_12.30_120621_NDD_SWP6000_2020.dss', 's0010_draftDCP_NDD_SWP6000_Beth_2020\\Model_Files\\DSS\\output\\4

,Prob_C_AMR004_LT_C_AMR004_MIF,Prob_C_AMR004_EQ_C_AMR004_MIF,Prob_C_TRN111_LT_C_TRN111_MIF,Prob_C_TRN111_EQ_C_TRN111_MIF,Prob_C_FTR029_LT_C_FTR029_MIF,Prob_C_FTR029_EQ_C_FTR029_MIF,Prob_C_MOK028_LT_C_MOK028_MIF,Prob_C_MOK028_EQ_C_MOK028_MIF,Prob_C_MCD005_LT_C_MCD005_MIF,Prob_C_MCD005_EQ_C_MCD005_MIF,...,Prob_C_SAC148_LT_C_SAC148_MIF,Prob_C_SAC148_EQ_C_SAC148_MIF,Prob_C_SAC122_LT_C_SAC122_MIF,Prob_C_SAC122_EQ_C_SAC122_MIF,Prob_C_FTR003_LT_C_FTR003_MIF,Prob_C_FTR003_EQ_C_FTR003_MIF,Prob_C_SAC049_LT_C_SAC049_MIF,Prob_C_SAC049_EQ_C_SAC049_MIF,Prob_C_SJR070_LT_C_SJR070_MIF,Prob_C_SJR070_EQ_C_SJR070_MIF
Scenario,,,,,,,,,,,,,,,,,,,,,
s0001,0.0,0.004167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,NaN,NaN,NaN
s0002,0.0,0.008333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,NaN,NaN,NaN
s0003,0.0,0.015833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.002500,NaN,NaN,NaN,NaN
s0004,0.0,0.015000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.001667,NaN,NaN,NaN,NaN
s0005,0.0,0.024167,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.001667,NaN,NaN,NaN,NaN
s0006,0.0,0.006206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,NaN,NaN,NaN
s0007,0.0,0.007092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,NaN,NaN,NaN
s0008,0.0,0.007092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,NaN,NaN,NaN
s0009,0.0,0.007092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,NaN,NaN,NaN


In [23]:
dss_names

['s0001_DCR2023_9.3.1_danube_hist\\Model_Files\\DSS\\output\\DCR2023_DV_9.3.1_Danube_Hist_v1.7.dss',
 's0002_DCR2023_9.3.1_danube_adj\\Model_Files\\DSS\\output\\DCR2023_DV_9.3.1_v2a_Danube_Adj_v1.8.dss',
 's0003_DCR2023_9.3.1_danube_cc50\\Model_Files\\DSS\\output\\DCR2023_DV_9.3.1_Danube_cc50_v1.8.dss',
 's0004_DCR2023_9.3.1_danube_cc75\\Model_Files\\DSS\\output\\DCR2023_DV_9.3.1_Danube_CC75_v1.8.dss',
 's0005_DCR2023_9.3.1_danube_cc95\\Model_Files\\DSS\\output\\DCR2023_DV_9.3.1_Danube_CC95_v1.8.dss',
 's0006_draftDCP_NDDOff_2020\\Model_Files\\DSS\\output\\0_DCR19_12.30_120621_NDDOff_2020.dss',
 's0007_draftDCP_NDD_SWP3000_2020\\Model_Files\\DSS\\output\\1_DCR19_12.30_120621_NDD_SWP3000_2020.dss',
 's0008_draftDCP_NDD_SWP4500_2020\\Model_Files\\DSS\\output\\2_DCR19_12.30_120621_NDD_SWP4500_2020.dss',
 's0009_draftDCP_NDD_SWP6000_2020\\Model_Files\\DSS\\output\\3_DCR19_12.30_120621_NDD_SWP6000_2020.dss',
 's0010_draftDCP_NDD_SWP6000_Beth_2020\\Model_Files\\DSS\\output\\4_DCR19_12.30_120

In [13]:
# Metrics (PDF)

variables = [
    "S_RESTOT_NOD_", "S_RESTOT_s",
    "S_SHSTA_", "S_OROVL_", "S_TRNTY_", "S_MELON_", "S_MLRTN_",
    "S_SHSTALEVEL5DV",
    "DEL_SWP_TOTAL_", "DEL_SWP_PMI_", "DEL_SWP_PAG_", "DEL_CVP_TOTAL_", "DEL_CVP_PAG_TOTAL_", "DEL_CVP_PSCEX_TOTAL_", "DEL_CVP_PRF_TOTAL_",
    "D_TOTAL_", "NDO_",
    "X2_PRV_KM_",
    "DEL_CVP_PSC_N_", "DEL_CVP_PAG_N_", "DEL_CVP_PAG_S_", "DEL_CVP_PRF_S_", "DEL_CVP_PMI_N_", "DEL_CVP_PMI_S_"
] 

In [14]:
metrics = {}

study_list = np.arange(0, len(dss_names))

for var in variables:
    if var in ["C_SAC041_", "C_SJR070_", "C_SAC000_", "C_SJR070_", "C_DMC000_TD_", "C_CAA003_TD_", "NDO_", "D_TOTAL_"]:
        units = "CFS"
    elif var == "X2_PRV_KM_":
        units = "KM"
    else:
        units = "TAF"

    if var in ["S_RESTOT_NOD_", "S_RESTOT_s"]: # 1 & 2
        metrics[f"Apr_{var}mnth_avg"] = mnth_avg(df, dss_names, var, 4, units) 
        metrics[f"Sept_{var}mnth_avg"] = mnth_avg(df, dss_names, var, 9, units) 

    if var in ["S_SHSTA_", "S_OROVL_"]: # 3 & 4
        metrics[f"Apr_{var}mnth_avg"] = mnth_avg(df, dss_names, var, 4, units) 

    if var in ["S_SHSTA_", "S_SHSTALEVEL5DV", "DEL_SWP_TOTAL_", "DEL_SWP_PMI_", "DEL_SWP_PAG_", "DEL_CVP_TOTAL_", "DEL_CVP_PAG_TOTAL_", "DEL_CVP_PSCEX_TOTAL_", "DEL_CVP_PRF_TOTAL_", "D_TOTAL_", "NDO_"]: # 8-14, 16 & 17
        metrics[f"{var}ann_avg"] = ann_avg(df, dss_names, var, units)

    if var in ["X2_PRV_KM_"]: # 18-21
        metrics[f"Fall_{var}ann_avg"] = ann_avg(df, dss_names, var, units, months=[9,10,11]).rename(columns={f"Ann_Avg_{var}{units}":f"Fall_Ann_Avg_{var}{units}"}) # might want to edit function to rename column
        metrics[f"Spring_{var}ann_avg"] = ann_avg(df, dss_names, var, units, months=[3,4,5]).rename(columns={f"Ann_Avg_{var}{units}":f"Spring_Ann_Avg_{var}{units}"}) # ^
        metrics[f"Apr_{var}compute_sd"] = compute_sd(df, var, "Oct_StDev_X2", [4], units).set_index(pd.Index([scenario[:5] for scenario in dss_names])) # there's probably a better way to do this
        metrics[f"Oct_{var}compute_sd"] = compute_sd(df, var, "Apr_StDev_X2", [10], units).set_index(pd.Index([scenario[:5] for scenario in dss_names])) # ^
        
for var, metric_df in metrics.items():
    write_out_dfs.append(metric_df)
    display(metric_df)

,Apr_Avg_S_RESTOT_NOD_TAF
s0001,9502.720622
s0002,9289.524626
s0003,8831.470114
s0004,8536.757009
s0005,8363.576594
s0006,8797.665441
s0007,8818.561157
s0008,8817.579762
s0009,8806.576365
s0010,8809.165562


,Sep_Avg_S_RESTOT_NOD_TAF
s0001,6800.449020
s0002,6570.594230
s0003,5655.789429
s0004,5292.440964
s0005,5134.175172
s0006,6152.457555
s0007,6193.778243
s0008,6194.621633
s0009,6187.818817
s0010,6188.742741


,Apr_Avg_S_RESTOT_sTAF
s0001,11511.439131
s0002,11303.156554
s0003,10903.099940
s0004,10550.181127
s0005,10321.439679
s0006,10641.042921
s0007,10662.103601
s0008,10661.234972
s0009,10650.272755
s0010,10652.850867


,Sep_Avg_S_RESTOT_sTAF
s0001,8537.728392
s0002,8283.957292
s0003,7223.204372
s0004,6785.620518
s0005,6569.599413
s0006,7733.253688
s0007,7774.708571
s0008,7775.646336
s0009,7768.880840
s0010,7769.795451


,Apr_Avg_S_SHSTA_TAF
s0001,4037.991051
s0002,3920.254867
s0003,3711.537404
s0004,3592.096895
s0005,3511.278046
s0006,4035.944251
s0007,4044.659226
s0008,4044.682619
s0009,4038.165765
s0010,4038.121174


...

,Ann_Avg_NDO_CFS
s0001,21293.470789
s0002,21893.727002
s0003,22047.182931
s0004,21389.026896
s0005,20485.437723
s0006,21149.723018
s0007,20541.031775
s0008,20408.485673
s0009,20338.511268
s0010,20332.168442


,Fall_Ann_Avg_X2_PRV_KM_KM
s0001,83.270578
s0002,83.335746
s0003,83.867571
s0004,83.622434
s0005,84.119572
s0006,82.993528
s0007,83.229084
s0008,83.240828
s0009,83.228258
s0010,83.232444


,Spring_Ann_Avg_X2_PRV_KM_KM
s0001,65.725578
s0002,65.517907
s0003,65.238535
s0004,65.683405
s0005,66.163310
s0006,65.889260
s0007,66.331013
s0008,66.433326
s0009,66.489764
s0010,66.489404


,Oct_StDev_X2
s0001,9.058433
s0002,9.545118
s0003,9.860989
s0004,9.954342
s0005,9.920891
s0006,8.959683
s0007,9.075646
s0008,9.094312
s0009,9.117835
s0010,9.118256


,Apr_StDev_X2
s0001,5.702494
s0002,6.037924
s0003,6.002244
s0004,6.398094
s0005,6.456646
s0006,5.758341
s0007,5.866146
s0008,5.878136
s0009,5.884412
s0010,5.877314


## Write out dataframes to single CSV

In [15]:
write_out_dfs.append(df_flow_probs)
combined_df = pd.concat(write_out_dfs, axis=1)
display(combined_df)
combined_df.to_csv(metrics_path + "/all_metrics_output.csv")

,All_Prob_S_SHSTA_flood,All_Prob_S_SHSTA_dead,Sept_Prob_S_SHSTA_dead,All_Prob_S_OROVL_flood,All_Prob_S_OROVL_dead,Sept_Prob_S_OROVL_dead,All_Prob_S_TRNTY_flood,All_Prob_S_TRNTY_dead,Sept_Prob_S_TRNTY_dead,All_Prob_S_SLUIS_CVP_flood,...,Prob_C_SAC148_LT_C_SAC148_MIF,Prob_C_SAC148_EQ_C_SAC148_MIF,Prob_C_SAC122_LT_C_SAC122_MIF,Prob_C_SAC122_EQ_C_SAC122_MIF,Prob_C_FTR003_LT_C_FTR003_MIF,Prob_C_FTR003_EQ_C_FTR003_MIF,Prob_C_SAC049_LT_C_SAC049_MIF,Prob_C_SAC049_EQ_C_SAC049_MIF,Prob_C_SJR070_LT_C_SJR070_MIF,Prob_C_SJR070_EQ_C_SJR070_MIF
s0001,0.333333,0.004167,0.01,0.215000,0.000000,0.00,0.000000,0.000000,0.00,0.078333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s0002,0.315000,0.017500,0.05,0.210000,0.000833,0.00,0.000833,0.000000,0.00,0.076667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s0003,0.222500,0.038333,0.10,0.165833,0.010000,0.02,0.000000,0.000000,0.00,0.055000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s0004,0.192500,0.052500,0.11,0.144167,0.016667,0.04,0.000000,0.000000,0.00,0.046667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s0005,0.175833,0.053333,0.11,0.133333,0.024167,0.06,0.000000,0.000000,0.00,0.041667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s0006,0.303333,0.003333,0.01,0.203333,0.000000,0.00,0.000000,0.004167,0.02,0.085000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s0007,0.301667,0.000000,0.00,0.201667,0.000000,0.00,0.000000,0.000000,0.00,0.087500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s0008,0.300833,0.000000,0.00,0.200000,0.000000,0.00,0.000000,0.000833,0.01,0.088333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s0009,0.300833,0.000000,0.00,0.201667,0.000000,0.00,0.000000,0.000833,0.01,0.090833,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
s0010,0.301667,0.000000,0.00,0.201667,0.000000,0.00,0.000000,0.000833,0.01,0.091667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
